In [20]:
import os, sys
sys.path.append('..')
from tqdm import tqdm

import torch
import torch.nn as nn
import matplotlib.pyplot as plt 

In [49]:
class TimeSeriesTransformerAutoencoder(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, seq_length, dropout=0.1):
        super(TimeSeriesTransformerAutoencoder, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)
        self.pos_encoder = nn.Parameter(torch.zeros(1, seq_length, model_dim))
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout, batch_first=True),
            num_layers=num_layers,
        )
        self.transformer_decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout, batch_first=True),
            num_layers=num_layers
        )
        self.fc_out = nn.Linear(model_dim, input_dim)

    def forward(self, x):
        x = self.embedding(x) + self.pos_encoder
        z = self.transformer_encoder(x)
        x_rec = self.transformer_decoder(z, z)
        return self.fc_out(x_rec)
    
    def encode(self, x):
        x = self.embedding(x) + self.pos_encoder
        return self.transformer_encoder(x)

In [50]:
from dataset.cycling_dataset import CyclingDataset
from torch.utils.data import DataLoader

train_dataset = CyclingDataset(
    file_path_acc='/Users/Aleyna/downloads/preprocessed/handlebar_acc_train.h5',
    file_path_gyro='/Users/Aleyna/downloads/preprocessed/handlebar_gyro_train.h5',
)

test_dataset = CyclingDataset(
    file_path_acc='/Users/Aleyna/downloads/preprocessed/handlebar_acc_test.h5',
    file_path_gyro='/Users/Aleyna/downloads/preprocessed/handlebar_gyro_test.h5',
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [51]:
model = TimeSeriesTransformerAutoencoder(input_dim=6, model_dim=16, num_heads=4, num_layers=4, seq_length=100)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

train_losses = []
val_losses = []

for epoch in range(1):
    model.train()
    for x, _ in tqdm(train_loader):
        x_rec = model(x)
        loss = torch.sqrt(criterion(x_rec, x)) # RMSE (reconstruction error)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    model.eval()
    with torch.no_grad():
        for i, (x, _) in enumerate(test_loader):
            x_rec = model(x)
            loss = torch.sqrt(criterion(x_rec, x))
            val_losses.append(loss.item())

    torch.save(model.state_dict(), 'transformer_autoencoder.pth')

100%|██████████| 154/154 [01:43<00:00,  1.49it/s]


In [53]:
model.encode(torch.randn(1, 100, 6)).shape

torch.Size([1, 100, 16])